In [1]:
import gym
env = gym.make('Marvin-v0')

In [2]:
init_observation = env.reset()
done = False
i = 0
r_sum = 0
while not done:
    observation, reward, done, _ = env.step(env.action_space.sample())
    i += 1
    r_sum += reward
print(f"Episode end after {i} iterations with reward = {r_sum}")

Episode end after 170 iterations with reward = -133.88062215148472


In [3]:
env.action_space.low, env.action_space.high

(array([-1., -1., -1., -1.], dtype=float32),
 array([1., 1., 1., 1.], dtype=float32))

In [4]:
env.observation_space.low, env.observation_space.high

(array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf], dtype=float32),
 array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       dtype=float32))

In [5]:
from multilayer_perceptron.mlp.nn import *

In [6]:
nn = NeuralNetwork('mse', 'adam', random_state=42)
nn.add(Dense(3, 'relu'))
nn.add(Dense(4, 'tanh'))
nn.compile(env.observation_space.shape[0])

In [7]:
nn.summary()

|Dense(24, 3, relu)                	|	75
|Dense(3, 4, tanh)                 	|	16
Total number of parameters:		 91
Total number of trainable params:	 91


In [8]:
params = nn.get_params()
params

[[array([[ 0.01355253,  0.01612121,  0.02178583,  0.00441266, -0.00191006,
           0.00918371, -0.00345347,  0.0115566 ,  0.00353245,  0.02575615,
           0.00704381, -0.02149307,  0.01023077,  0.00324853, -0.00298232,
           0.00340093, -0.02068245,  0.00790484,  0.00320478, -0.0063767 ,
          -0.00841096, -0.01773392,  0.00879417,  0.00732145],
         [ 0.01687626,  0.01189374,  0.01282178,  0.01566677,  0.00179239,
          -0.00729483, -0.00475361, -0.00361311,  0.00259407, -0.02435597,
           0.01179155, -0.00211171,  0.0015397 ,  0.00159293, -0.00293244,
           0.00060637, -0.00040726,  0.0032989 , -0.00821363, -0.00732216,
           0.01700943, -0.0074014 , -0.01194719,  0.00668488],
         [ 0.00084694, -0.00268574, -0.00911532,  0.00287715,  0.00043487,
          -0.00073875,  0.00759685, -0.00161963,  0.00889213, -0.00174674,
           0.02144417, -0.02454866,  0.01052558, -0.00041242, -0.01031607,
           0.02009771,  0.00604797, -0.0063798 , 

In [12]:
params[0][1][1] = 1

In [ ]:
original_params = [[]]
for layer_params in params:
    original_params.np.copy()

In [13]:
params

[[array([[ 0.01355253,  0.01612121,  0.02178583,  0.00441266, -0.00191006,
           0.00918371, -0.00345347,  0.0115566 ,  0.00353245,  0.02575615,
           0.00704381, -0.02149307,  0.01023077,  0.00324853, -0.00298232,
           0.00340093, -0.02068245,  0.00790484,  0.00320478, -0.0063767 ,
          -0.00841096, -0.01773392,  0.00879417,  0.00732145],
         [ 0.01687626,  0.01189374,  0.01282178,  0.01566677,  0.00179239,
          -0.00729483, -0.00475361, -0.00361311,  0.00259407, -0.02435597,
           0.01179155, -0.00211171,  0.0015397 ,  0.00159293, -0.00293244,
           0.00060637, -0.00040726,  0.0032989 , -0.00821363, -0.00732216,
           0.01700943, -0.0074014 , -0.01194719,  0.00668488],
         [ 0.00084694, -0.00268574, -0.00911532,  0.00287715,  0.00043487,
          -0.00073875,  0.00759685, -0.00161963,  0.00889213, -0.00174674,
           0.02144417, -0.02454866,  0.01052558, -0.00041242, -0.01031607,
           0.02009771,  0.00604797, -0.0063798 , 